### Tensorflow CNN
---

#### 卷积网络概述 
---
1. 卷积神经网络
    * 卷积层 : 
        1. 拥有至少一个卷积层的神经网络 `tf.nn.conv2d`
        2. 卷积层的目的在于，计算输入的张量 `f` 和一个可以学习的卷积核 `g` 的卷积，生成该层的输出(输出特征图 `feature map`)
    * 原理 : 
        1. 根据猴子的纹状细胞的研究发现，如果存在某种模式的刺激，某些细胞层会释放信号
        2. 这些模式被称之为是 **滤波器** 或者 **卷积核**，通过梯度下降法训练卷积核是的和训练数据可以有效的匹配是算法的核心
    * 网络架构 : 
        1. 卷积层 : `tf.nn.conv2d`
        2. 非线性变换层 : `tf.nn.relu`
        3. 最大池化层 : `tf.nn.max_pool`
        4. 全连接层 : `tf.matmul` 手动搭建全连接网络
    * 输入数据
        1. 专用输入流水线处理
        2. 读入数据的标准格式
            ```python
            [image_batch_size, image_height, image_width, image_channels]
            ```
        3. 在实际的 **调试过程中** 我们可以使用 **假数据** 实现对数据的调试和测试操作，只需要用假数据当做测试数据接到输入管线上即可
    * 作用机理
        
        1. 降维 : 步长 `strides` 策略可以降低维数，一般来说对于 `batch_size` 和 `channels` 的策略都是选择不跳过的(步长是1),但是降维的话需要考虑对 `height`, `width` 的维度步长扩大
        
        2. 边界填充 : `Tensorflow` 会自动的使用 0 来进行填充，或者说当不选择填充的时候会抛出一个边界异常
            
            1. `SAME` 填充 : 卷积的输入和输出的尺寸相同(信道不同)，跨越图像的时候，不考虑滤波器的尺寸，缺失的像素使用 0 填充，实际扫过的像素数目多于图片的像素数目
            2. `VALID` 填充 : 卷积核计算的时候尽量不会超过图片的尺寸，某些情况下存在填充的现象
       
2. 卷积函数  
    `tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=True, data_format='NHWC', name=None)`
        
        1. `input` : 输入 `tensor`， `[batch_size, height. width, channels]`
        2. `filter` : 卷积核， `[height, width, in_channels, out_channels]`
        3. `strides` : 步长，长为 4 的张量，代表每一个对应的 `input` 数据上的窗口移动步长(一般来说，对于`batch_size`上的步长都是1,不要漏过数据)
        4. `padding` : 填充 `SAME` 不变填充, `VALID` 缩减填充
        5. `name` : `Op` 名称  
        6. `data_format` : 数据格式              
            1. NHWC : 默认格式 
            
            [batch, in_height, in_width, in_channels]
            
            2. NCHW
            
            [batch, in_channels, in_height, in_width]
            
            3. 定义

<table>
<tr><td>数据格式符号</td><td>定义</td></tr>
<tr><td>N</td><td>Batch size</td></tr>
<tr><td>H</td><td>Height</td></tr>
<tr><td>W</td><td>Width</td></tr>
<tr><td>C</td><td>Channel</td></tr>
</table>

In [1]:
sess = tf.Session()

# 卷积操作实验，返回一个张量，第一个通道不变，第二个通道是第一个通道的2倍的张量
# [2, 2, 2, 1], 2 - batch, 2 - 高, 2 - 宽, 1 - 通道
input_batch = tf.constant([
    [
        [[0.0], [1.0]],
        [[2.0], [3.0]]
    ], 
    [
        [[2.0], [4.0]],
        [[6.0], [8.0]]
    ]
])

# [1, 1, 1, 2], 1 - 高, 1 - 宽, 1 - 通道, 2 - 卷积核的数目
kernel = tf.constant([
    [
        [[1.0, 2.0]]
    ]
])

print(sess.run(tf.shape(input_batch)), sess.run(tf.shape(kernel)))

# 卷积操作,步长都是1的SAME卷积
conv2d = tf.nn.conv2d(input_batch, kernel, strides=[1, 1, 1, 1], padding='SAME')

print(sess.run(conv2d))
# [2, 2, 2, 2], 2 / 2 / 2 - 不变填充， 2 - 卷积核个数
print(sess.run(tf.shape(conv2d)))

[2 2 2 1] [1 1 1 2]
[[[[  0.   0.]
   [  1.   2.]]

  [[  2.   4.]
   [  3.   6.]]]


 [[[  2.   4.]
   [  4.   8.]]

  [[  6.  12.]
   [  8.  16.]]]]
[2 2 2 2]


#### 基础操作 
---
1. 卷积核
    1. 在计算机视觉中，卷积核常常用于识别数字图像中的重要特征，当滤波器(卷积核)感兴趣的特征存在的时候，滤波器会使用特定的模式突出特征，这些特征很多，在图像中比如 (颜色，色调，边缘检测等等图像的属性特征)
    2. 在算法的训练过程中，不断的调整卷积核从而适应不同的特征抽取，学习到好的滤波器才是卷积神经网络的主要目标
    
2. 网络层次架构
    
    1. **CNN** 中并不是仅仅只限于只写常用的层结构，实际上其他的网络层结构也可以混合设计，但是传统的有效的 CNN 还是只用这几种特殊的层结构的，有些特殊的层结构可以有效的避免 CNN 的过拟合的情况发生
    2. 其他的有用的卷积操作
        
        1. `tf.nn.depthwise_conv2d`
            
            可以辅助构建 `Inception` 架构的网络
            
        2. `tf.nn.separable_conv2d`
            
            1. 大规模计算可以不损失准确率加速
            2. 小规模计算可以加速计算但是准确率有所降低
            
        3. `tf.nn.conv2d_transpose`
            
            反卷积层，可以有效的构建出 `FCN` 全卷积网络
            
    3. 激活函数
        
        1. 非线性的激活函数可以刻画更加复杂的变化和输入特征
        2. 激活函数应该存在的特点
            
            1. 函数单调 : 梯度下降寻优可能
            2. 函数可微 : 保证梯度的存在， ReLU 函数虽然在 0 处不存在梯度，但是我们可以重定义这一点的梯度
        3. 常用激活函数
            
            1. `tf.nn.relu` : 
                1. 引入信息的损失但是性能非常的优秀
                2. 削弱梯度消失的情况，梯度的取值是 `[0, +无穷]`
                3. 但是存在较大的学习率的时候，容易收到饱和的神经元的影响
                    [解释链接](https://www.v2ex.com/t/340003)
                
            2. `tf.sigmoid` / `tf.nn.sigmoid` : 
                1. 输出的范围缩减到 `[0.0, 1.0]` 范围之内
                2. 容易受梯度消失的影响
                3. **但是只接收浮点数，不接受整数**
                
            3. `tf.tanh` : 
                1. 类似 `sigmoid` 函数
                2. 值域是 `[-1.0, 1.0]`, 可以考虑成是 `sigmoid` 的变换
                
    4. 正则化 `dropout` 　
        
        `tf.nn.dropout(x, keep_prob, name)`
        
        1. 内部使用的是反向随机失活
        2. `x` 中元素存在 `keep_prob` 的概率被放大 `1 / keep_prob`, 否则输出 0.0 表示失活
    
    5. 池化
        
        1. 输入降采样，避免过拟合，提高计算性能
        2. 常用池化
            
            1. `tf.nn.max_pool(value, kernel_size, strides, padding, data_format, name)` : 最大池化
            2. `tf.nn.avg_pool` : 平均池化
            
    6. 归一化 **`LRN`**
        1. `tf.nn.local_response_normalization` : 局部响应归一化
        2. 对局部神经元的活动创建竞争机制，使得其中响应比较大的值变得相对更大，并抑制其他反馈较小的神经元，增强了模型的泛化能力
        
    7. 高级层次结构
        
        1. `tf.contrib.layers.convolution2d` : 使用可选的 `batch` 归一化层添加一个 (1,2,3)D 卷积层,返回输出特征图
            1. ` tf.contrib.layers.convolution2d(inputs, num_outputs, kernel_size, stride=1, padding='SAME', data_format=None, rate=1, activation_fn=tf.nn.relu, normalizer_fn=None, trainable=True, scope=None)`
            
            2. 参数解析 **这里的 `N` 代表的是纯图像的维度，不含通道和 `batch`**
                1. `inputs`: 输入变量，是一个 `N+2` 维的 `Tensor`(包含有 `batch` 和 `channels` 维度的 `N+2` 维张量)
                    1. 类型要求是一个 `Tensor` ，而我们一般训练的数据都是常量(比如mnist，load以后得到是python的数据类型，不是tf的)，所以需要把用tf的方法做一下转换，比如 `tf.reshape`
                    2. 为什么是 `N+2` 维呢，比如图像，除了宽度和高度，实际上还有样本数量(batch)和通道数量(channel)(如 `RGB3` 通道)，所以多了2维。
                    3. `inputs` 的格式，由 `date_format` 这个参数来觉得，比如2维，有 `NHWC` 和 `NCHW` 两种

                2. `num_outputs`: 卷积 `filter` 的数量，或者说提取的特征数量，比如 5,10
                3. `kernel_size`: 卷积核的大小，是N个参数的list，比如二维图像，可以时候[10,10]，如果参数值相同，用一个整数来表示也可以,代表所有的 N 个维度的值都是一样的
                4. `stride`: 卷积步长，同样是N个参数的序列，或者都相等的话，用一个整数来表示表示所有的维度步长都是一样的，默认是1.
                5. `padding`: 字符串格式，默认 `SAME` ，可选 `VALID`
                6. `data_format`: 字符串，指定inputs的格式
                    1. 一维数据：”NWC” (default) and “NCW”
                    2. 二维数据：”NHWC” (default) and “NCHW”
                    3. 三维数据：”NDHWC”
                    
                    也就是，不指定的话，通道数都是最后一个参数,批大小都是第一个参数
                7. `activation_fn`: 激活函数，默认relu
                8. `normalizer_fn`: 可以添加 `batch` 归一化函数
                9. `trainable`：  
                    `If True also add variables to the graph collection GraphKeys.TRAINABLE_VARIABLES`
                    默认是True,允许训练
                10. `weight` / `bias` 的初始化(`weight_initializer`, ...)可以通过类似 `tf.random_normal` 的初始化函数传入
        2. `tf.contrib.layers.fully_connected` : 全连接层高度封装
            1. `tf.contrib.layers.fully_connected(inputs, num_outputs, activation_fn, normalizer_fn=None, trainable=True, scope=None)`
                自动创建内部参数，添加一层全连接层
            
            2. 参数解析
                1. `inputs` : 输入最好是 `[batch_size, depth]` 的形式，如果不是在传递之前需要 `tf.reshape` 展平,输入维度小于2报错
                2. `num_outpus` : 输出单元的神经元个数
                3. `activation_fn` : 指定给隐藏层，默认是ReLU
                4. `normalizer_fn` : 可以添加 `batch` 归一化
                5. `trainable` : 和上面的一样
                
        3. `tf.contrib.layers.dropout` : 随机失活
            1. `tf.contrib.layers.dropout(inputs, keep_prob=0.5, noise_shape=None, is_training=True, outputs_collections=None, scope=None)`
                   给输入的张量加入随机失活，存在有 `keep_prob` 概率 `x` 中的元素会被放大 `1 / keep_prob` 倍，否则变成0表示失活，从而保持输出的数值的规模不变(反向随机失活)
            2. 参数解析
                1. `input` : 输入张量，即将被传送给节点 `tf.nn.dropout`
                2. `keep_prob` : 和 `x` 大小相同的张量，表示随机失活的概率
                3. `is_training` : `True` 加入 `dropout`, `False`直接返回输入 `input`
                    ```python
                    # 这样的使用方式决定是否使用 dropout 
                    ... is_training = (mode == tf.contrib.learn.ModelKeys.TRAIN) ...
                    ```


In [2]:
# relu 
print(sess.run([tf.range(-2, 4), tf.nn.relu(tf.range(-2, 4))]))

# sigmoid
print(sess.run([tf.range(-2, 4), tf.sigmoid(tf.range(-2.0, 4.0))]))

# tanh
print(sess.run([tf.range(-2, 4), tf.tanh(tf.to_float(tf.range(-2, 4)))]))

# dropout
print(sess.run([tf.range(-2, 4), tf.nn.dropout(tf.to_float(tf.range(-2, 4)), keep_prob=0.5)]))

[array([-2, -1,  0,  1,  2,  3], dtype=int32), array([0, 0, 0, 1, 2, 3], dtype=int32)]
[array([-2, -1,  0,  1,  2,  3], dtype=int32), array([ 0.11920292,  0.26894143,  0.5       ,  0.7310586 ,  0.88079703,
        0.95257413], dtype=float32)]
[array([-2, -1,  0,  1,  2,  3], dtype=int32), array([-0.96402758, -0.76159418,  0.        ,  0.76159418,  0.96402758,
        0.99505472], dtype=float32)]
[array([-2, -1,  0,  1,  2,  3], dtype=int32), array([-0., -0.,  0.,  2.,  4.,  0.], dtype=float32)]


In [5]:
# 池化, batch, height, width, channel [1, 3, 3, 1]
layer_input = tf.constant([
    [
        [[1.0], [0.2], [1.5]],
        [[0.1], [1.2], [1.4]],
        [[1.1], [0.4], [0.4]]
    ]
])

kernel_size = [1, 3, 3, 1]
print(sess.run(tf.shape(layer_input)))
max_pool = tf.nn.max_pool(layer_input, kernel_size, [1, 1, 1, 1], 'VALID')
sess.run(max_pool)

[1 3 3 1]


array([[[[ 1.5]]]], dtype=float32)

In [2]:
# tf.contrib.layers.convolution2d
# 注意，tensorlfow对与颜色的控制浮点数表示范围在 [0,1] 之间，下面的示例并不是正确的表示图像的方式，应该使用 tf.image.convert_image_dtype
image_input = tf.constant([
    [
        [[0., 0., 0.], [255., 255., 255.], [254., 0., 0.]],
        [[0., 191., 0.], [3., 108., 233.], [0., 191., 0.]],
        [[254., 0., 0.], [255., 255., 255.], [0., 0., 0.]]
    ]
])

# batch = 1, height = width = 3, channels = 3
print(sess.run(tf.shape(image_input)))

conv2d = tf.contrib.layers.convolution2d(image_input, num_outputs = 4, 
                                          kernel_size = (1, 1), activation_fn = tf.nn.relu,
                                          stride = (1, 1))

# 高级层中定义了变量，需要初始化
sess.run(tf.global_variables_initializer())
sess.run([conv2d, tf.shape(conv2d)])

[1 3 3 3]


[array([[[[   0.        ,    0.        ,    0.        ,    0.        ],
          [ 107.40837097,    0.        ,    0.        ,   60.8921814 ],
          [   0.        ,    0.        ,    0.        ,  188.04109192]],
 
         [[   0.        ,  120.6844635 ,    0.        ,    5.7305522 ],
          [ 155.06538391,    0.        ,   50.30088806,    0.        ],
          [   0.        ,  120.6844635 ,    0.        ,    5.7305522 ]],
 
         [[   0.        ,    0.        ,    0.        ,  188.04109192],
          [ 107.40837097,    0.        ,    0.        ,   60.8921814 ],
          [   0.        ,    0.        ,    0.        ,    0.        ]]]], dtype=float32),
 array([1, 3, 3, 4], dtype=int32)]

In [15]:
# dropout 高级层

input = tf.contrib.layers.dropout([1., 2., 3., 4., 5.], keep_prob=0.5, is_training = True)
print(sess.run(input))
input = tf.contrib.layers.dropout([1., 2., 3., 4., 5.], keep_prob=0.5, is_training = False)
print(sess.run(input))

[  0.   4.   0.   0.  10.]
[ 1.  2.  3.  4.  5.]


In [15]:
# 全连接层 高级层
input = tf.range(0, 10)
input = tf.cast(tf.reshape(input, [-1, 10]), dtype=tf.float32)
result = tf.contrib.layers.fully_connected(input, num_outputs=3)
sess.run(tf.global_variables_initializer())   # 高级层内部存在有参数，必须初始化全部参数
print(sess.run(result))

[[ 8.91926479  5.50948334  0.        ]]


#### 图像和 Tensorflow
[TFRecord参考连接](https://www.cnblogs.com/upright/p/6136265.html)

---
1. 简述
    1. Tensorflow 支持将图像作为神经网络的输入，支持 (JPG, PNG), 支持颜色空间描述 (RGB, RGBA)
    2. 但是图像使用操作容易的话，需要对图像的加载尺寸做出调整
    3. RGB 颜色空间 : `[Red, Green, Blue]`
    4. 所有的图像都需要从磁盘上加载到内存中做出训练
    5. 对于超出内存的错误，小注意，可能是提取的图片数据过大导致的
   
2. 加载图像
    1. `tf.train.string_input_producer`
    2. `tf.WholeFileReader` / `tf.read_file(filename, name)`
    3. `tf.image.decode_jpeg` : 解码文件
    
3. 图像格式
    1. 图像如果过大，导致图像的张量占用过大的内存或者显存，增加训练的时间，并且而数据过大的话容易是的学习到一些无用的信息，增加过拟合性影响泛化能力
    2. 处理格式支持 : 其他的格式的图像可以很方便的转化成这几种图像格式
        1. `tf.image.decode_gif`
        2. `tf.image.decode_jpeg` : 无法处理透明度，因为 "伪影"　的存在，请尽量减少对 jpeg 图像的操作
        3. `tf.image.decode_png` : 可以处理透明度，图片质量闭 jpeg 好，但是文件的数据量比较大
        4. `tf.image.decode_bmp`
        5. `tf.image.decode_image` : 对上述函数的一个封装，简便操作，无需决定图像的格式
4. `TFRecord`
    1. 作用
        1. 将二进制数据和对应的标签存入同一个文件中，需要预处理成 TFRecord 文件保存在磁盘中
        2. 没有压缩过，可以快速的加载到内存中
        3. 可以降低训练时候编写的代码的复杂性,使用 TFReocrd 是推荐的使用做法
        
    2. 相关操作
        1. 构建一个 TFRecord 文件 writer  
            `tf.python_io.TFRecordWriter(path/to/the/tfrecord/file)`
        2. 在写入文件之前，需要对 `numpy` 数据转变成对应的字节数组二进制存储  
            `numpy.array.tobytes()`
           
        3. 构建 TFRecord 文件必要组件
            1. `tf.train.Features` ： Features里包含Feature（这里没s）的字典
            2. `tf.train.Feature` ： 一个数据项,**存在和字典一样的数据读取API接口**
            3. `tf.train.BytesList` : 字节类型的数据项
            4. `tf.train.Int64List` : 整数类型的数据项
            5. `tf.train.Example` ： 协议内存块，基本的，一个Example中包含Features
            
            ```python
            example = tf.train.Example(features=tf.train.Features(feature={
            # 加入标签
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            # 加入数据
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        }))
            ```
        4. 数据写入
            ```python
            # 序列化数据成二进制的数据，写入到文件中
            writer.write(example.SerializeToString())
            ```
            
        5. 数据加载
            1. `tf.TFReordReader` : TFRecord 文件读取器
            2. `tf.parse_single_example` : 解析 TFRecord 文件
            3. `tf.FixedLenFeature([], tf.string)`
            4. `tf.FixedLenFeature([], tf.int64)`
            
            ```python
            # serialized_example 代表的是已经用读取器从文件名队列中读取的 TFRecord 文件，生成的对象具有和字典类似的 API
            # features 中的键和我们写入文件的时候的键是对应的
            features = tf.parse_single_example(serialized_example,
                                       features={
                                           # 要和文件中的数据类型对应
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })
            ```
    3. **`Laze Loading` 的注意点**
        1. 如果遇到我们处理 TFRecord 文件出现处理速度越来越慢的情况，需要仔细的留心了，可能出现了 `laze loading` 的陷阱，导致计算图的容量越来越大
        2. 实际上，如果我们在 `Python` 的循环中，不断的使用修改我们的计算图的话，实际上我们会发现我们每一次运行 `sess.run` 的速度都会降低，这是因为我们如果不断的在循环中向计算图加入节点的话，计算图的规模会变得越来越大，从而导致之后的计算速度越来越慢
        3. 问题的原因在于我们在**运行时的 `session` **中加入了计算节点到计算图中

In [2]:
# 加载图像，转换成 TFRecord 文件
with tf.Session() as sess:
    fq = tf.train.string_input_producer(['/home/lantian/Downloads/StanfordDog/Images/n02085620-Chihuahua/n02085620_10074.jpg'], num_epochs=1, shuffle=False)
    sess.run(tf.local_variables_initializer())
    image_reader = tf.WholeFileReader()
    _, image_file = image_reader.read(fq)
    image = tf.image.decode_jpeg(image_file)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord=coord)
    # batch = 1,忽略
    # print(sess.run([image, tf.shape(image)]))
    
    image_label = b'\x01'    # 假标签，随便加的
    image_loaded = sess.run(image)
    image_bytes = image_loaded.tobytes()
    imh, imw, imc = image_loaded.shape
    
    print(imh, imw, imc, type(image_loaded))
    
    writer = tf.python_io.TFRecordWriter('./test.tfrecord')
    
    # 样本文件
    example = tf.train.Example(features = tf.train.Features(feature = {
        'label' : tf.train.Feature(bytes_list = tf.train.BytesList(value = [image_label])),
        'image' : tf.train.Feature(bytes_list = tf.train.BytesList(value = [image_bytes]))
    }))
    
    # 写入 TFRecord 文件,不要在 jupyter 中打开这个文件，编码错误，****记住必须写完要关闭文件读写器不然会一直追加到文件中****
    writer.write(example.SerializeToString())
    writer.close()
    

500 333 3 <class 'numpy.ndarray'>


In [4]:
# 数据加载　TFRecord
with tf.Session() as sess:
    fq = tf.train.string_input_producer(['./test.tfrecord'], num_epochs=1, shuffle=False)
    sess.run(tf.local_variables_initializer())
    tf_record_reader = tf.TFRecordReader()
    _, tf_record_ser = tf_record_reader.read(fq)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
    
    try :
        tf_record_example = tf.parse_single_example(tf_record_ser, 
                                                   features = {
                                                       'label' : tf.FixedLenFeature([], tf.string),
                                                       'image' : tf.FixedLenFeature([], tf.string)
                                                   })
        # 使用 tf.uint8 形式表示图片像素
        tf_record_image = tf.decode_raw(tf_record_example['image'], tf.uint8)
        tf_record_image = tf.reshape(tf_record_image, [imh, imw, imc])
        tf_record_label = tf.cast(tf_record_example['label'], tf.string)
        # 加载成功
        print(sess.run([tf_record_label, tf.shape(tf_record_image)]))
        # 展开成卷积网络接收的维度
        print(sess.run(tf.reshape(tf_record_image, [500, 333, 3])))
    except Exception as e:
        pass
    finally:
        coord.request_stop()
    coord.join(threads)

[b'\x01', array([500, 333,   3], dtype=int32)]


#### 图像操作
图像预处理

---
为什么需要图像操作
    
    1. 在实际中我们的数据集中的图片可能存在有大量的无关信息，影响我们的网络的性能，我们需要使用预处理的方式将我们需要学习的主要特征突显出来，这就是我们引入图像操作的主要原因
    2. 我们还可以用图像操作对已有的图像进行其他的操作从而增加数据集的多样性，降低过拟合的程度
    3. 常用的图像操作有 : 裁剪，缩放，灰度调整等
    4. 可以考虑使用Python的其他的图像处理的库执行这一步操作，但是Tensorflow中已经集成了一些这样操作处理的方法   

#### 颜色
---

1. 灰度图像在大多数计算机视觉任务中是合理的，了解图像的信息并不是很依赖颜色信息，缩减颜色空间可以加快处理速度,但是有时候需要进行颜色空间的变换
2. 灰度图转换
    `tf.image.rgb_to_grayscale(image)`
3. HSV 空间转换
    1. 色彩，饱和度，灰度值构成 HSV 空间，比较贴合人类的感知
    2. `tf.image.rgb_to_hsv`

4. LAB 空间转换 : `Python` 的第三方库 `python-colormath`